# Letter Recognition using SVM
**Author - Girish Shirke,** 
**Date - 06/03/2020**


Dataset is a collection of various hand written letter which people usually fill out in forms

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix


/home/girish/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/girish/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
data = pd.read_csv('letter-recognition.txt', sep=',')

In [3]:
data.head()

,T,2,8,3,5,1,8.1,13,0,6,6.1,10,8.2,0.1,8.3,0.2,8.4
0,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
1,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
2,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
3,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
4,S,4,11,5,8,3,8,8,6,9,5,6,6,0,8,9,7


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19999 entries, 0 to 19998
Data columns (total 17 columns):
T      19999 non-null object
2      19999 non-null int64
8      19999 non-null int64
3      19999 non-null int64
5      19999 non-null int64
1      19999 non-null int64
8.1    19999 non-null int64
13     19999 non-null int64
0      19999 non-null int64
6      19999 non-null int64
6.1    19999 non-null int64
10     19999 non-null int64
8.2    19999 non-null int64
0.1    19999 non-null int64
8.3    19999 non-null int64
0.2    19999 non-null int64
8.4    19999 non-null int64
dtypes: int64(16), object(1)
memory usage: 2.6+ MB
